In [2]:
%load_ext autoreload
%autoreload 2

In [8]:
import json
import pandas as pd
import os
import sys
import cv2
sys.path.append('..')
import matplotlib.pyplot as plt
from fidelity_check import find_diff_writes

### Check the assumption of archive's writes are always a subset of live

### Fidelity check

In [39]:
dirr = '../revert_rewrite/writes/lni.wa.gov_12080'
fixedIdx = json.load(open(os.path.join(dirr, 'result_log.json'), 'r'))["fixedIdx"]
print("fixedIdx", fixedIdx)
left = 'initial'
right = f'exception_{fixedIdx}'
issue, (left_u, right_u) = find_diff_writes.fidelity_issue(dirr, left, right)
print(issue)
print(json.dumps(left_u, indent=2))
print(json.dumps(right_u, indent=2))


fixedIdx -1


FileNotFoundError: [Errno 2] No such file or directory: '../revert_rewrite/writes/lni.wa.gov_12080/exception_-1_elements.json'

### Screenshot similarity

In [35]:
dirr = '../revert_rewrite/writes/fs.usda.gov_11099'
fixedIdx = json.load(open(os.path.join(dirr, 'result_log.json'), 'r'))["fixedIdx"]
initial = os.path.join(dirr, 'initial.png')
exception = os.path.join(dirr, f'exception_{fixedIdx}.png')

def compare_screenshot(live_img, archive_img):
    if not os.path.exists(live_img) or not os.path.exists(archive_img):
        return -1
    img1 = cv2.imread(live_img)
    img2 = cv2.imread(archive_img)
    height = min(img1.shape[0], img2.shape[0])
    width = min([img1.shape[1], img2.shape[1]])
    img1 = img1[:height,:width,:]
    img2 = img2[:height,:width,:]
    diff = img1 - img2
    total = img1.shape[0]*img1.shape[1]*img1.shape[2]
    same = np.count_nonzero(diff == 0)
    return same / total, diff

simi, diff = compare_screenshot(initial, exception)
print(simi)
cv2.imwrite(os.path.join(dirr, 'diff.png'), diff)

0.9981350405808179


True

In [ ]:
dirs = os.listdir('../record_replay/writes')
num_diff, num_diff_writes = 0, 0
total = 0
for dirr in dirs:
    if not os.path.exists(f'../record_replay/writes/{dirr}/live_elements.json') or not os.path.exists(f'../record_replay/writes/{dirr}/archive_elements.json'):
        continue
    key_writes = find_diff_writes.locate_key_writes(f'../record_replay/writes/{dirr}')
    live_additional = [len(w['key_related_writes']) for w in key_writes['live']]
    archive_additional = [len(w['key_related_writes']) for w in key_writes['archive']]

    total += 1
    if len(live_additional) > 0 or len(archive_additional) > 0:
        num_diff += 1
    # if sum(archive_additional) > 0:
    #     num_diff_writes += 1

    ive_unique_elements, archive_unique_elements = find_diff_writes.find_diff_elements(f'../record_replay/writes/{dirr}')
    diff_writes = find_diff_writes.collect_diff_writes(f'../record_replay/writes/{dirr}')
    if len(diff_writes['archive']) > 0:
        print(len(diff_writes['archive']), len(archive_unique_elements))
        num_diff_writes += 1
print("Total", total)
print("Number of different layout trees", num_diff)
print("Number of archive's additional writes", num_diff_writes)

### fidelity check on a set of loads

In [14]:
def valid_dir(dirr):
    valid = os.path.exists(f'{dirr}/live_elements.json') and os.path.exists(f'{dirr}/archive_elements.json')
    return valid

base = '../record_replay/writes'
data = json.load(open(f'{base}/fidelity_check_metadata.json', 'r'))
results = []
for url, info in list(data.items()):
    dirr = info['directory']
    if not valid_dir(f"{base}/{dirr}"):
        continue
    issue, _ =  find_diff_writes.fidelity_issue(f"{base}/{dirr}")
    issue_screenshot, simi = find_diff_writes.fidelity_issue_screenshot(f"{base}/{dirr}")
    results.append({
        'directory': dirr,
        'issue': issue,
        'issue_screenshot': issue_screenshot,
        'screenshot_similarity': simi
    })
json.dump(results, open('fidelity_check.json', 'w+'), indent=2)